<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Hot-Dog" data-toc-modified-id="Hot-Dog-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Hot Dog</a></span><ul class="toc-item"><li><span><a href="#Загрузка-изображений" data-toc-modified-id="Загрузка-изображений-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка изображений</a></span></li><li><span><a href="#Model-From-Scratch" data-toc-modified-id="Model-From-Scratch-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Model From Scratch</a></span></li><li><span><a href="#Feature-Extraction" data-toc-modified-id="Feature-Extraction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Feature Extraction</a></span></li><li><span><a href="#Error-Analysis" data-toc-modified-id="Error-Analysis-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Error Analysis</a></span></li></ul></li><li><span><a href="#Dog" data-toc-modified-id="Dog-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dog</a></span><ul class="toc-item"><li><span><a href="#Feature-Extraction" data-toc-modified-id="Feature-Extraction-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Feature Extraction</a></span></li><li><span><a href="#Fine-Tuning" data-toc-modified-id="Fine-Tuning-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Fine Tuning</a></span></li><li><span><a href="#Evaluation" data-toc-modified-id="Evaluation-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Evaluation</a></span></li></ul></li></ul></div>

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
plt.style.use("ggplot")
import seaborn as sns
from sklearn.metrics import roc_auc_score

from IPython.core.display import clear_output

# Hot Dog

https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/data

<img src="img/silicon_valley.jpg">

In [ ]:
# !mkdir ./data

# !wget https://raw.githubusercontent.com/shestakoff/sphere-ml-intro/master/2020/lecture11-cnn/data/seefood.zip -O ./data/seefood.zip

In [ ]:
!unzip data/seefood.zip -d data

In [ ]:
# !tree -d data/seefood/

## Загрузка изображений

In [ ]:
TRAIN_PATH = "data/seefood/train"
TEST_PATH = "data/seefood/test"
SEED = 42

IMG_SIZE = 224
batch_size = 16
class_names = ["not_hot_dog", "hot_dog"]


train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_PATH,
    labels="inferred",
    class_names=class_names,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size
)

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TRAIN_PATH,
    labels="inferred",
    class_names=class_names,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size
)

test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory=TEST_PATH,
    labels="inferred",
    class_names=class_names,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=batch_size
)

In [ ]:
N_ROWS = 4
N_COLS = batch_size // N_ROWS

fig, ax = plt.subplots(nrows=N_ROWS, ncols=N_COLS, figsize=(10, 10))

for images, targets in train_ds.take(1):
    images = images.numpy().astype("uint8")
    for i in range(N_ROWS):
        for j in range(N_COLS):
            ax[i, j].set_title(class_names[targets[i * N_COLS + j]])
            ax[i, j].imshow(images[i * N_COLS + j])
            ax[i, j].axis("off")
plt.show()

## Model From Scratch

In [ ]:
cnn = tf.keras.models.Sequential([
    tf.keras.layers.Input([IMG_SIZE, IMG_SIZE, 3], dtype=tf.float32),
    tf.keras.layers.experimental.preprocessing.Resizing(width=64, height=64),
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=100, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
],
name="cnn")

In [ ]:
cnn.summary()

In [ ]:
class MetricsCallback(tf.keras.callbacks.Callback):
    
    def __init__(self, metrics_name):
        super().__init__()
        self.loss = []
        self.val_loss = []
        self.metrics = []
        self.val_metrics = []
        self.metrics_name = metrics_name

    def on_epoch_end(self, epoch, logs=None):
        
        self.loss.append(logs["loss"])
        self.val_loss.append(logs["val_loss"])
        self.metrics.append(logs[self.metrics_name])
        self.val_metrics.append(logs[f"val_{self.metrics_name}"])
        
        clear_output(wait=True)
        
        plt.figure(figsize=(12,5))
        plt.subplot(121)
        plt.plot(self.loss, label="loss")
        plt.plot(self.val_loss, label="val_loss")
        plt.legend()

        plt.subplot(122)
        plt.plot(self.metrics, label=self.metrics_name)
        plt.plot(self.val_metrics, label=f"val_{self.metrics_name}")
        plt.legend()
        plt.show()

In [ ]:
cnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=tf.keras.metrics.AUC(name='auc')
)

history = cnn.fit(
    train_ds, validation_data=val_ds, epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=20),
        tf.keras.callbacks.ReduceLROnPlateau(),
        MetricsCallback(metrics_name="auc")
    ]
)

In [ ]:
augmentation = tf.keras.models.Sequential(
    [
        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.3),
        tf.keras.layers.experimental.preprocessing.RandomTranslation(height_factor=0.1, width_factor=0.1)
    ],
    name="augmentation"
)

In [ ]:
plt.figure(figsize=(9,9))
for image, label in train_ds.unbatch().take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        image_tensor = tf.expand_dims(image, axis=0)
        augmented_tensor = augmentation(image_tensor)
        plt.imshow(augmented_tensor[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
aug_cnn = tf.keras.models.Sequential([
    tf.keras.layers.Input([224, 224, 3], dtype=tf.float32),
    tf.keras.layers.experimental.preprocessing.Resizing(width=64, height=64),
    tf.keras.layers.experimental.preprocessing.Rescaling(scale=1./255),
    augmentation,
    tf.keras.layers.Conv2D(filters=10, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=50, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.Conv2D(filters=100, kernel_size=3, padding="same", activation="relu"),
    tf.keras.layers.MaxPool2D(pool_size=2),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dropout(rate=0.4),
    tf.keras.layers.Dense(50, activation="relu"),
    tf.keras.layers.Dense(1, activation="sigmoid")
],
name="aug_cnn")

In [ ]:
aug_cnn.summary()

In [ ]:
aug_cnn.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=tf.keras.metrics.AUC(name='auc')
)

history_aug = aug_cnn.fit(
    train_ds, validation_data=val_ds, epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(patience=10),
        tf.keras.callbacks.ReduceLROnPlateau(),
        MetricsCallback(metrics_name="auc")
    ]
)

## Feature Extraction

In [ ]:
backbone = tf.keras.applications.ResNet50(include_top=False, weights='imagenet')

In [ ]:
backbone.layers

In [ ]:
for image, _ in train_ds:
    outputs = backbone(images)

In [ ]:
outputs

In [ ]:
backbone.trainable

In [ ]:
backbone.trainable = False

In [ ]:
i = tf.keras.layers.Input([224, 224, 3], dtype=tf.float32)
x = tf.keras.applications.resnet50.preprocess_input(i)
x = backbone(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.4)(x)
x = tf.keras.layers.Dense(units=1, activation="sigmoid")(x)
model = tf.keras.Model(inputs=[i], outputs=[x])

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=tf.keras.metrics.AUC(name='auc')
)

history_fe = model.fit(
    train_ds, validation_data=val_ds, epochs=10,
    callbacks=[
        tf.keras.callbacks.ReduceLROnPlateau(),
        MetricsCallback(metrics_name="auc")
    ]
)

## Error Analysis

In [ ]:
predictions = []
targets = []
images = []

for image, target in test_ds:
    images.append(image.numpy())
    predictions.append(model(image).numpy())
    targets.append(target.numpy())

In [ ]:
images = np.concatenate(images)
targets = np.concatenate(targets)
predictions = np.concatenate(predictions).reshape(-1)

In [ ]:
roc_auc_score(
    y_true=targets,
    y_score=predictions
)

In [ ]:
top_k = 10

false_positive_ids = ((targets == 0) * predictions).argsort()[::-1][:top_k]

positive_ids = np.arange(targets.shape[0])[(targets == 1)]

false_negative_ids = positive_ids[predictions[(targets == 1)].argsort()[:top_k]]

In [ ]:
for id_ in false_positive_ids:
    plt.title(f"not_hot_dog, score: {predictions[id_]:.2f}")
    plt.imshow(images[id_].astype("uint8"))
    plt.axis("off")
    plt.show()

In [ ]:
for id_ in false_negative_ids:
    plt.title(f"hot_dog, score: {predictions[id_]:.2f}")
    plt.imshow(images[id_].astype("uint8"))
    plt.axis("off")
    plt.show()

# Dog

In [ ]:
IMG_SIZE = 224
batch_size = 64

(ds_train, ds_test), ds_info = tfds.load(
    name="stanford_dogs", split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

In [ ]:
ds_train = ds_train.map(
    lambda image, label: (tf.image.resize(image, [IMG_SIZE, IMG_SIZE]), label)
)
ds_test = ds_test.map(
    lambda image, label: (tf.image.resize(image, [IMG_SIZE, IMG_SIZE]), label)
)

In [ ]:
N_ROWS = 3
N_COLS = 3


def format_label(label):
    string_label = ds_info.features["label"].int2str(label)
    return string_label.split("-")[1]

plt.figure(figsize=(9,9))

for i, (image, label) in enumerate(ds_train.take(N_ROWS * N_COLS)):
    ax = plt.subplot(N_ROWS, N_COLS, i + 1)
    plt.imshow(image.numpy().astype("uint8"))
    plt.title("{}".format(format_label(label)))
    plt.axis("off")

In [ ]:
augmentation = tf.keras.models.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomRotation(factor=0.25),
        tf.keras.layers.experimental.preprocessing.RandomFlip(),
        tf.keras.layers.experimental.preprocessing.RandomContrast(factor=0.25),
        tf.keras.layers.Lambda(lambda x: tf.clip_by_value(x, 0, 255))
    ],
    name="augmentation"
)

In [ ]:
plt.figure(figsize=(9,9))
for image, label in ds_train.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        image_tensor = tf.expand_dims(image, axis=0)
        augmented_tensor = augmentation(image_tensor)
        plt.imshow(augmented_tensor[0].numpy().astype("uint8"))
        plt.axis("off")

In [ ]:
ds = ds_train.shuffle(buffer_size=10_000)

ds_train = ds.skip(2_000).batch(batch_size)

ds_val = ds.take(2_000).batch(batch_size)

## Feature Extraction

In [ ]:
backbone = tf.keras.applications.ResNet50(include_top=False)
backbone.trainable = False

In [ ]:
i = tf.keras.layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
x = augmentation(i)
x = backbone(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax")(x)

model = tf.keras.Model(inputs=[i], outputs=[x])    

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer="adam", 
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

In [ ]:
fe_history = model.fit(
    ds_train, validation_data=ds_val, epochs=10, 
    callbacks=[MetricsCallback(metrics_name="accuracy")]
)

## Fine Tuning

In [ ]:
for layer in backbone.layers[-20:]:
    if not isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = True

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=4e-5), 
    loss="sparse_categorical_crossentropy",
    metrics="accuracy"
)

In [ ]:
ft_history = model.fit(
    ds_train, validation_data=ds_val, epochs=10, 
    callbacks=[MetricsCallback(metrics_name="accuracy")]
)

In [ ]:
plt.figure(figsize=(12,5))

plt.subplot(121)
plt.plot(range(1, 11), fe_history.history["val_accuracy"]) 
plt.plot(range(11, 21), ft_history.history["val_accuracy"])

plt.subplot(122)
plt.plot(range(1, 11), fe_history.history["val_loss"]) 
plt.plot(range(11, 21), ft_history.history["val_loss"])
plt.show();

## Evaluation

In [ ]:
N_ROWS = 4
N_COLS = 4

for images, labels in ds_test.batch(N_ROWS * N_COLS).take(1):

    predicted_labels = tf.argmax(model(images), axis=1)
  
    plt.figure(figsize=(20,20))
    for i, (label, predicted_label) in enumerate(zip(labels, predicted_labels)):
        ax = plt.subplot(N_ROWS, N_COLS, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))
        plt.title(f"predicted: {format_label(predicted_label)}\n label: {format_label(label)}")
        plt.axis("off")
    plt.show()

In [ ]:
conf_matrix = np.zeros((NUM_CLASSES, NUM_CLASSES))

for images, labels in ds_test.batch(128):
    predicted_labels = tf.argmax(model(images), axis=1)

    for label, predicted_label in zip(labels, predicted_labels):
        conf_matrix[predicted_label, label] += 1

In [ ]:
plt.figure(figsize=(12,12))
sns.heatmap(conf_matrix);

In [ ]:
true_label = []
predicted_label = []
count = []

for i in range(NUM_CLASSES):
    for j in range(NUM_CLASSES):
        if i != j and conf_matrix[i,j] > 0:
            predicted_label.append(format_label(i))
            true_label.append(format_label(j))
            count.append(conf_matrix[i,j])

In [ ]:
(
    pd.DataFrame({"label": true_label, "predicted": predicted_label, "count": count})
    .sort_values("count", ascending=False)
    .head(10)
)